In [9]:
import pandas as pd

# Data sources:

https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions?select=RAW_recipes.csv

https://www.kaggle.com/datasets/shuyangli94/foodcom-recipes-with-search-terms-and-tags

https://www.kaggle.com/datasets/irkaal/foodcom-recipes-and-reviews

# Read data

Download all data sources and unpack then in src directory

In [2]:
# read cvs file "PP_recipes.csv" and print first 10 rows
raw_recipes = pd.read_csv('src/RAW_recipes.csv')
raw_recipes = raw_recipes[['id', 'minutes', 'submitted', 'tags']]
raw_recipes.head(2)

,id,minutes,submitted,tags
0,137739,55,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course..."
1,31490,30,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course..."


In [3]:
recipes_with_ingredients = pd.read_csv('src/recipes_w_search_terms.csv')
recipes_with_ingredients = recipes_with_ingredients[['id', 'name', 'ingredients_raw_str', 'ingredients', 'steps', 'description']]
recipes_with_ingredients.head(2)

,id,name,ingredients_raw_str,ingredients,steps,description
0,96313,Grilled Garlic Cheese Grits,"[""4 cups water"",""1 cup uncooked old f...","['water', 'grits', 'salt', 'cheddar cheese', '...","['I a sauce pan, bring water to a boil; slowly...","We love grits, this is another good way to ser..."
1,232037,Simple Shrimp and Andouille Jambalaya,"[""1 medium onion, chopped coarse "",""1 m...","['onion', 'red bell pepper', 'garlic cloves', ...","['In a food processor, pulse the onion, red pe...","Simple, easy and very tasty for when you are i..."


In [4]:
recipes_big = pd.read_parquet('src/recipes_with_imgs.parquet', engine='fastparquet')
print(recipes_big.columns)
recipes_big = recipes_big[['RecipeId', 'AuthorId', 'AuthorName', 'ReviewCount']]
recipes_big.head(2)

Index(['RecipeId', 'Name', 'AuthorId', 'AuthorName', 'CookTime', 'PrepTime',
       'TotalTime', 'DatePublished', 'Description', 'Images', 'RecipeCategory',
       'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts',
       'AggregatedRating', 'ReviewCount', 'Calories', 'FatContent',
       'SaturatedFatContent', 'CholesterolContent', 'SodiumContent',
       'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent',
       'RecipeServings', 'RecipeYield', 'RecipeInstructions'],
      dtype='object')


,RecipeId,AuthorId,AuthorName,ReviewCount
0,38.0,1533,Dancer,4.0
1,39.0,1567,elly9812,1.0


In [5]:
recipes_tokenized = pd.read_csv('src/PP_recipes.csv')
recipes_tokenized = recipes_tokenized[['id', 'calorie_level', 'ingredient_ids']]
recipes_tokenized.head(2)

,id,calorie_level,ingredient_ids
0,424415,0,"[389, 7655, 6270, 1527, 3406]"
1,146223,0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."


# Prepare recipes base

In [6]:
# merge interactions_count with recipes based in id
recipes_full = pd.merge(raw_recipes, recipes_with_ingredients, left_on='id', right_on='id', how='left')
# merge interactions_count with recipes based in id
recipes_full = pd.merge(recipes_full, recipes_tokenized, left_on='id', right_on='id', how='left')
# merge with recipes_big
recipes_full = pd.merge(recipes_full, recipes_big, left_on='id', right_on='RecipeId', how='left')

recipes_full = recipes_full.dropna(subset=['AuthorId', 'AuthorName'])

In [7]:
# sort recipes by review count and take top 1000
base_recipes = recipes_full.sort_values(by=['ReviewCount'], ascending=False)
base_recipes.dropna()

# drop recipes where ingredient_ids are nan
base_recipes = base_recipes.dropna(subset=['ingredient_ids'])
base_recipes = base_recipes.head(5000)


base_recipes

,id,minutes,submitted,tags,name,ingredients_raw_str,ingredients,steps,description,calorie_level,ingredient_ids,RecipeId,AuthorId,AuthorName,ReviewCount
22168,2886,65,1999-09-26,"['time-to-make', 'course', 'main-ingredient', ...",Best Banana Bread,"[""1/2 cup butter, softened "",""1 cup g...","['butter', 'granulated sugar', 'eggs', 'banana...","['Remove odd pots and pans from oven.', 'Prehe...",You'll never need another banana bread recipe ...,1.0,"[840, 3355, 2499, 342, 63, 335, 6270, 7449]",2886.0,1762.0,lkadlec,2273.0
213826,27208,545,2002-05-03,"['weeknight', 'time-to-make', 'course', 'main-...",To Die for Crock Pot Roast,"[""1 (4 -5 lb) beef roast, any kind "",""1 (...","['beef roast', 'brown gravy mix', 'dried itali...","['Place beef roast in crock pot.', 'Mix the dr...","Amazing flavor, and so simple! No salt needed ...",1.0,"[469, 784, 2200, 2200, 7655]",27208.0,28201.0,yooper,1692.0
66687,89204,243,2004-04-16,"['time-to-make', 'course', 'main-ingredient', ...",Crock-Pot Chicken With Black Beans &amp; Cream...,"[""4 -5 boneless chicken breasts, frozen ...","['boneless chicken breasts', 'black beans', 'c...","['Take 4-5 frozen, yes, frozen, boneless chick...",I love this Crock-Pot chicken recipe for two r...,2.0,"[1252, 553, 1803, 6265, 1910]",89204.0,137839.0,Jen Santiago,1657.0
62431,39087,25,2002-09-02,"['30-minutes-or-less', 'time-to-make', 'course...",Creamy Cajun Chicken Pasta,"[""2 boneless skinless chicken breast hal...","['boneless skinless chicken breast halves', 'l...",['Place chicken and Cajun seasoning in a bowl ...,N'awlin's style of chicken with an updated Alf...,2.0,"[1254, 4416, 905, 840, 6335, 3670, 6926, 6270,...",39087.0,30534.0,Lorac,1586.0
22388,67256,75,2003-07-24,"['weeknight', 'time-to-make', 'course', 'main-...",Best Ever Banana Cake With Cream Cheese Frosting,"[""1 1/2 cups bananas, mashed, ripe "",""2 ...","['bananas', 'lemon juice', 'flour', 'baking so...","['Preheat oven to 275°F (135C).', 'Grease and ...",This is one of (if not) the BEST banana cake I...,2.0,"[342, 4253, 2683, 335, 6270, 840, 6906, 2499, ...",67256.0,82367.0,Rhonda J,1409.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123547,442185,30,2010-11-16,"['30-minutes-or-less', 'time-to-make', 'course...",Light and Fluffy Biscuits,"[""4 cups all-purpose flour"",""1 teaspoon...","['all-purpose flour', 'salt', 'baking powder',...","['Combine the flour, salt, baking powder and s...","If you like biscuits, I am sure you will like ...",1.0,"[63, 6270, 332, 6906, 855, 2499, 4717]",442185.0,1728457.0,skeptic777,25.0
92715,348549,17,2009-01-09,"['30-minutes-or-less', 'time-to-make', 'course...",Glazed Carrots With Maple Syrup and Sesame Seeds,"[""1 lb carrot, peeled and cut into batons...","['carrot', 'butter', 'honey', 'salt', 'water',...",['Put the carrots into a medium pan with half ...,"Wanting a quick carrot dish last night, I turn...",0.0,"[1093, 840, 3723, 6270, 7655, 6416]",348549.0,323186.0,Karen Elizabeth,25.0
227203,54787,38,2003-02-24,"['60-minutes-or-less', 'time-to-make', 'course...",WHOOPIE PIES - the REAL Deal - Lancaster Co. R...,"[""1 cup oil"",""2 cups brown sugar"",""4...","['oil', 'brown sugar', 'eggs', 'flour', 'salt'...","['Cream shortening, sugar and eggs.', 'Add van...",MMMMMMMmmmmmmmmmm! If you've never had a whoop...,2.0,"[4987, 800, 2499, 2683, 6270, 335, 1584, 4717,...",54787.0,41706.0,WJKing,25.0
34388,17449,20,2002-01-18,"['30-minutes-or-less', 'time-to-make', 'course...",Campbell's Chicken & Broccoli Alfredo,"[""8 ounces linguine"",""1 cup fresh, fr...","['linguine', 'broccoli florets', 'butter', 'bo...",['Cook linguine according to package direction...,NaN,2.0,"[4416, 761, 840, 1252, 1920, 4717, 5180, 5319]",17449.0,20754.0,RecipeNut,25.0


# Ingredients table

In [10]:
ingredients = pd.read_pickle('src/ingr_map.pkl')

# create new ingredients table data, that stores only processed and id columns off current ingredients data
ingredients_processed = ingredients[['id', 'replaced']]
ingredients_processed.columns = ['id', 'name']

# remove repetitions in ingredients_processed
ingredients_processed = ingredients_processed.drop_duplicates()
# save processed ingredients to csv file
ingredients_processed.to_csv('result/ingredients.csv', index=False)

ingredients_processed

,id,name
0,4308,lettuce
42,2744,french vanilla pudding and pie filling mix
43,6843,stove top stuffing mix
45,1910,cream cheese
86,1168,cheddar
...,...,...
11654,6702,soybean
11655,3318,goose
11656,47,ajwain
11657,750,brinjal


# Recipes-ingredients table

In [13]:
import ast
# create empty pandas dataframe
recipes_ingredients = pd.DataFrame()

to_drop = []
for index, row in base_recipes.iterrows():
    try:
        ingredients_raw = ast.literal_eval(row['ingredients_raw_str'])
        ingredients_ids = ast.literal_eval(row['ingredient_ids'])
        ingredients_names = ast.literal_eval(row['ingredients'])
    except:
        to_drop.append(index)
        continue

    if len(ingredients_names) > len(ingredients_ids):
        ingredients_names = ingredients_names[:len(ingredients_ids)]
    elif len(ingredients_names) < len(ingredients_ids):
        ingredients_ids = ingredients_ids[:len(ingredients_names)]

    if len(ingredients_raw) > len(ingredients_names):
        ingredients_raw = ingredients_raw[:len(ingredients_names)]
    elif len(ingredients_raw) < len(ingredients_names):
        ingredients_names = ingredients_names[:len(ingredients_raw)]
    
    ingredients_quantity = []
    for ingredient_raw, ingredient_name in zip(ingredients_raw, ingredients_names):
        if ingredient_name in ingredient_raw:
            quantity = ingredient_raw.split(ingredient_name)[0].strip()
            ingredients_quantity.append(quantity)
        else:
            ingredients_quantity.append('')
    # create dataframe with names and id's 
    ingredients_array = pd.DataFrame({'ingredient_id': ingredients_ids, 'quantity': ingredients_quantity, })

    # add row id column to tags_array
    ingredients_array['recipe_id'] = row['id']

    # add row to recipes_tags dataframe
    recipes_ingredients = pd.concat([recipes_ingredients, ingredients_array], ignore_index=True)

# drop recipes with index in to_drop
base_recipes = base_recipes.drop(to_drop)

# change the order of columns in recipes_ingredients
recipes_ingredients = recipes_ingredients[['recipe_id', 'ingredient_id', 'quantity' ]]

# drop duplicate rows, where recipe_id and ingredient_id are the same
recipes_ingredients = recipes_ingredients.drop_duplicates(subset=['recipe_id', 'ingredient_id'])

recipes_ingredients.to_csv('result/recipes_ingredients.csv', index=False)
print(len(to_drop), len(base_recipes))
recipes_ingredients.head(2)

0 4912


,recipe_id,ingredient_id,quantity
0,2886,840,1/2 cup
1,2886,3355,1 cup


# Tags table

In [14]:
forbidden_tags = ['time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 
'north-american', 'equipment', '4-hours-or-less', '5-ingredients-or-less', 'beef', 'crock-pot-slow-cooker',
 'roast-beef', 'taste-mood', 'poultry', 'chicken', 'chicken-breasts', 'corn', 'southern-united-states',
  'bananas', 'holiday-event', 'southwestern-united-states', 'apples', 'beans', 'eggs', 'beef-ribs',
   'black-beans', 'brown-rice', 'refrigerator', 'tuna', 'grapes', 'tomatoes', 'salmon', 'pork']

In [15]:
import ast
# create empty pandas dataframe
tags = pd.DataFrame()

# iterate for every row in recipes_with_interactions_1
for index, row in base_recipes.iterrows():
    tags_array = ast.literal_eval(row['tags'])
    tags_array = filter(lambda x: x not in forbidden_tags, tags_array)
    # add tags to tags dataframe
    tags = pd.concat([tags, pd.DataFrame(tags_array)], ignore_index=True)

# remove repetitions in tags
tags = tags.drop_duplicates()
# name first column in tags "tag_name"
tags.columns = ['name']
tags['id'] = tags.index

tags = tags[['id', 'name']]
# add id column to tags

tags.to_csv('result/tags.csv', index=False)
tags.head(2)

,id,name
0,0,breads
1,1,fruit


# Recipes-tags table

In [16]:
# create empty pandas dataframe
recipes_tags = pd.DataFrame()

# iterate for every row in recipes_with_interactions_1
for index, row in base_recipes.iterrows():
    tags_array = ast.literal_eval(row['tags'])
    tags_array = filter(lambda x: x not in forbidden_tags, tags_array)

    # find tag id for every tag name in tags_array
    tags_ids = tags[tags['name'].isin(tags_array)]['id'].values

    tags_ids_df = pd.DataFrame(tags_ids, columns=['tag_id'])

    # add row id column to tags_array
    tags_ids_df['recipe_id'] = row['id']

    # add row to recipes_tags dataframe
    recipes_tags = pd.concat([recipes_tags, tags_ids_df], ignore_index=True)

# change the order of columns in recipes_tags
recipes_tags = recipes_tags[['recipe_id', 'tag_id']]
recipes_tags.to_csv('result/recipes_tags.csv', index=False)

recipes_tags.head(2)

,recipe_id,tag_id
0,2886,0
1,2886,1


# Recipes-steps table

In [17]:
recipes_steps = pd.DataFrame()

for index, row in base_recipes.iterrows():
    steps = ast.literal_eval(row['steps'])
    
    # create dataframe with names and id's 
    steps = pd.DataFrame({'step_content': steps, 'step_number': range(1, len(steps) + 1)})

    # add row id column to tags_array
    steps['recipe_id'] = row['id']

    # add row to recipes_tags dataframe
    recipes_steps = pd.concat([recipes_steps, steps], ignore_index=True)

# change the order of columns in recipes_ingredients
recipes_steps = recipes_steps[['recipe_id', 'step_number', 'step_content']]

recipes_steps.to_csv('result/recipes_steps.csv', index=False)
recipes_steps

,recipe_id,step_number,step_content
0,2886,1,Remove odd pots and pans from oven.
1,2886,2,Preheat oven to 350º / 180º.
2,2886,3,Cream together butter and sugar.
3,2886,4,Add eggs and crushed bananas.
4,2886,5,Combine well.
...,...,...,...
36736,26233,4,Stir in everything else and heat to boiling.
36737,26233,5,Put in a 1 1/2 quart casserole and cover.
36738,26233,6,Bake for 45 minutes.
36739,26233,7,Remove bay leaves.


# Users table

In [18]:
from passlib.context import CryptContext
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

# find distinct contributor_id in recipes_table
users = base_recipes[['AuthorId', 'AuthorName']].drop_duplicates()

# change the name of the first column "id"
users.columns = ['id', 'username']
users['id'] = users['id'].astype(int)

users['username'] = users['username'].str.replace(" ", "")
# add email column which is equal to id
users['email'] = users['username'].str.lower() + '@unknown.com'
users['hashed_password'] =  pwd_context.hash('password123')
users['last_login'] =  '2023-01-20'

users.to_csv('result/users.csv', index=False)
users

,id,username,email,hashed_password,last_login
22168,1762,lkadlec,lkadlec@unknown.com,$2b$12$WizDGSv.ICDcxRCy7jKAdesLu67HQx/sYDtmmk4...,2023-01-20
213826,28201,yooper,yooper@unknown.com,$2b$12$WizDGSv.ICDcxRCy7jKAdesLu67HQx/sYDtmmk4...,2023-01-20
66687,137839,JenSantiago,jensantiago@unknown.com,$2b$12$WizDGSv.ICDcxRCy7jKAdesLu67HQx/sYDtmmk4...,2023-01-20
62431,30534,Lorac,lorac@unknown.com,$2b$12$WizDGSv.ICDcxRCy7jKAdesLu67HQx/sYDtmmk4...,2023-01-20
22388,82367,RhondaJ,rhondaj@unknown.com,$2b$12$WizDGSv.ICDcxRCy7jKAdesLu67HQx/sYDtmmk4...,2023-01-20
...,...,...,...,...,...
225193,353491,CleanPlateClub,cleanplateclub@unknown.com,$2b$12$WizDGSv.ICDcxRCy7jKAdesLu67HQx/sYDtmmk4...,2023-01-20
99246,47579,Peeps,peeps@unknown.com,$2b$12$WizDGSv.ICDcxRCy7jKAdesLu67HQx/sYDtmmk4...,2023-01-20
205436,58552,NHNichole,nhnichole@unknown.com,$2b$12$WizDGSv.ICDcxRCy7jKAdesLu67HQx/sYDtmmk4...,2023-01-20
6247,135566,LainieBug,lainiebug@unknown.com,$2b$12$WizDGSv.ICDcxRCy7jKAdesLu67HQx/sYDtmmk4...,2023-01-20


# Recipes table

In [23]:
recipes_table = base_recipes.drop(['steps', 'ingredients', 'ingredient_ids', 'ingredients_raw_str', 'tags', 'RecipeId', 'AuthorName'], axis=1)

# change column name call ReviewCount to reactions
recipes_table = recipes_table.rename(columns={'ReviewCount': 'likes', 'submitted': 'date_added', 'name': 'title', 'AuthorId': 'contributor_id'})
recipes_table = recipes_table[['id', 'contributor_id', 'title', 'minutes', 'description',  'date_added', 'likes', 'calorie_level']]

recipes_table['description'] = recipes_table['description'].str.capitalize()
recipes_table['contributor_id'] = recipes_table['contributor_id'].astype(int)
recipes_table['calorie_level'] = recipes_table['calorie_level'].astype(int)
recipes_table['likes'] = recipes_table['likes'].astype(int)

recipes_table.to_csv('result/recipes.csv', index=False)

recipes_table

,id,contributor_id,title,minutes,description,date_added,likes,calorie_level
22168,2886,1762,Best Banana Bread,65,You'll never need another banana bread recipe ...,1999-09-26,2273,1
213826,27208,28201,To Die for Crock Pot Roast,545,"Amazing flavor, and so simple! no salt needed ...",2002-05-03,1692,1
66687,89204,137839,Crock-Pot Chicken With Black Beans &amp; Cream...,243,I love this crock-pot chicken recipe for two r...,2004-04-16,1657,2
62431,39087,30534,Creamy Cajun Chicken Pasta,25,N'awlin's style of chicken with an updated alf...,2002-09-02,1586,2
22388,67256,82367,Best Ever Banana Cake With Cream Cheese Frosting,75,This is one of (if not) the best banana cake i...,2003-07-24,1409,2
...,...,...,...,...,...,...,...,...
123547,442185,1728457,Light and Fluffy Biscuits,30,"If you like biscuits, i am sure you will like ...",2010-11-16,25,1
92715,348549,323186,Glazed Carrots With Maple Syrup and Sesame Seeds,17,"Wanting a quick carrot dish last night, i turn...",2009-01-09,25,0
227203,54787,41706,WHOOPIE PIES - the REAL Deal - Lancaster Co. R...,38,Mmmmmmmmmmmmmmmmm! if you've never had a whoop...,2003-02-24,25,2
34388,17449,20754,Campbell's Chicken & Broccoli Alfredo,20,NaN,2002-01-18,25,2
